In [94]:
import os
import time
import pandas as pd
from datetime import datetime

In [13]:
# 数据路径
data_path = r"F:\collection_spb_info\XJSH\ALL_DATA\ALL_RESULT\ALL_TABLE\52_贵州省_result_20250109.xlsx"
trrz_path = r"F:\collection_spb_info\XJSH\ALL_DATA\ALL_TRRZ\贵州省\黔南布依族苗族自治州\522701_都匀市\522701_trrz_info_20250220_1121.xlsx"

In [113]:
df = pd.read_excel(data_path)

In [114]:
use_col = ['ydbh','dwjd','dwwd','dwgc','ydlb','yypbh','jcjg','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz']

In [115]:
# col_dict = {'ydbh':"YDBH",'dwjd':"DWJD",'dwwd':"DWWD",'dwgc':"DWGC",'ydlb','yypbh','jcjg','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz'}

In [116]:
dy_df = df[df['xjdm']==522701]

In [117]:
dy_df = dy_df[use_col]

In [118]:
len(dy_df['ydbh'].unique()),len(dy_df['yypbh'].unique()),dy_df.shape[0]

(1159, 1362, 1394)

In [119]:
# 去除yypbh结尾不为10和11的
dy_df['yypbh'] = dy_df['yypbh'].astype(str)
filter_dy_df = dy_df[dy_df['yypbh'].str.endswith('10') | dy_df['yypbh'].str.endswith('11')]
len(filter_dy_df['ydbh'].unique()),len(filter_dy_df['yypbh'].unique()),filter_dy_df.shape[0]

(1159, 1159, 1188)

In [120]:
# 去除yypbh重复的数据，只保留第一个
re_filter_dy_df = filter_dy_df.drop_duplicates(subset=['yypbh'], keep='first')
len(re_filter_dy_df['ydbh'].unique()),len(re_filter_dy_df['yypbh'].unique()),re_filter_dy_df.shape[0]

(1159, 1159, 1159)

In [121]:
# 计算gzchd,(gzchd_x,gzchd_y,两列不为空和不为0的值，如果为空和0，则取0)
def cal_gzchd(row):
    try:
        if float(row['gzchd_x']) != 0:
            return float(row['gzchd_x'])
        elif float(row['gzchd_y']) != 0:
            return float(row['gzchd_y'])
        else:
            return 0
    except:
        return 0

In [122]:
result_col = use_col = ['ydbh','dwjd','dwwd','dwgc','yypbh','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz']

In [123]:
result_df = re_filter_dy_df[result_col]

In [124]:
# 获取耕作层厚度
result_df['gzchd'] = result_df.apply(cal_gzchd,axis=1)
# 删除gzchd_x,gzchd_y
result_df = result_df.copy()
result_df.drop(columns=['gzchd_x','gzchd_y'],inplace=True)

C:\Users\Runker\AppData\Local\Temp\ipykernel_55748\3945413382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['gzchd'] = result_df.apply(cal_gzchd,axis=1)


In [125]:
# 连接土壤容重信息
trrz_df = pd.read_excel(trrz_path)
trrz_df = trrz_df[['ydbh','trrzpjz']]
trrz_df.rename(columns={'ydbh':'YDBH','trrzpjz':'TRRZ'},inplace=True)


In [126]:
# 'YDBH', 'DWJD', 'DWWD', 'DWGC', 'TRZD', 'PH', 'CEC', 'OM', 'TN', 'TP', 'TK', 'TSE', 'AP', 'SK', 'AK', 'HG', 'AS2', 'PB', 'CD', 'CR', 'TRRZ', 'GZCHD', 'YXTCHD', 'FSXTL', 'FSXYL', 'FSXTS', 'FSXTZ'
col_dict = {'ydbh':'YDBH','dwjd':'DWJD','dwwd':'DWWD','dwgc':'DWGC',
            'yypbh':'YYPBH','yjz':'OM','ph':'PH','ylzjhl':'CEC',
            'qlin':'TP','qjia':'TK','qxi':'TSE','yxlin':'AP','qdan':'TN',
            'sxjia':'AK','hxjia':'SK','zgong':'HG',
            'zshen':'AS2','zqian':'PB','zge':'CD',
            'zge2':'CR','jxzc1trzd':'TRZD',
            'gzchd':'GZCHD','yxtchd':'YXTCHD',
            'fsxtl':'FSXTL','fsxyl':'FSXYL',
            'fsxts':'FSXTS','fsxtz':'FSXTZ'}

In [127]:
# 重命名列
result_end_df = result_df.copy()
result_end_df.rename(columns=col_dict,inplace=True)

In [128]:
# 连接容重信息
result_end_join_df = pd.merge(result_end_df,trrz_df,on='YDBH',how='left')

In [129]:
# 导出
save_path = r'G:\soil_property_result\dys\table'
os.makedirs(save_path,exist_ok=True)
result_end_join_df['YDBH'] = result_end_join_df['YDBH'].astype(str)
result_end_join_df.to_excel(os.path.join(save_path,f"result_all_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"),index=False)
result_end_join_df.to_excel(os.path.join(save_path,f"result_ana_df_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"),index=False)